# ScaffoldGraph: Cytoscape Visualization

ScaffoldGraph contains a utility to visualize scaffoldgraphs interactively in Jupyter using the [ipycytoscape](https://github.com/QuantStack/ipycytoscape) module.

To use the utility ipycytoscape must be installed:

`pip install ipycytoscape` or `conda install -c conda-forge ipycytoscape`

In [10]:
import scaffoldgraph as sg
import os

# Import the visualization utility.
from scaffoldgraph.vis.notebook import cytoscape

The visualizer can be used for all scaffold graph types. Here we will use the `ScaffoldTree`.

In [27]:
sdf_file = os.path.dirname(sg.__file__).replace('scaffoldgraph', 'examples/example.sdf')

# Construct a ScaffoldTree.
tree = sg.ScaffoldTree.from_sdf(sdf_file, progress=True)

# Print a quick summary.
print(sg.utils.summary(tree))


Type: ScaffoldTree
Number of molecule nodes: 199
Number of scaffold nodes: 450
Number of edges: 584
Max hierarchy: 6
Min hierarchy: 1



Initialize the visualizer with the scaffold tree. Note that custom styles can also be specified, allowing customisation of the visualisation. e.g. colouring molecule nodes according to activity. For more information on styles see the [ipycytoscape documentation](https://ipycytoscape.readthedocs.io/en/latest/index.html). Here we will just use the default style provided.

In [6]:
visualizer = cytoscape.CytoscapeVisualizer(tree)  # initialize.

The visualizer allows the user to draw the whole graph or just a subset of the graph starting from a molecule or scaffold node. Given that the tree can often be very large visualizing small subsets if often a better idea.

* visualizer.draw (draw the whole graph)
* visualizer.draw_for_molecule (draw a molecule and its constituent scaffolds)
* visualizer.draw_for_scaffold (draw a scaffold and its parent and/or children nodes)

When drawing scaffold trees edges are annotated with the rule used during prioritization.

In [19]:
# Draw graph for PubChem ID '1201903'.
widget = visualizer.draw_for_molecule('1201903')

# Widget properties can be modified after construction. 
widget.layout.height = '600px'

# display the widget.
widget

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'nodeSpacing': 50, 'edgeLengthVal': 50}, cytoscape_style=[{…

Using the hierarchy 1 scaffold from the last query ('1201903'), lets draw the subgraph originating from this scaffold ('c1ccncc1').

When creating visualizations from scaffold there is an additional option `traversal` specifying the direction of subgraph creation {'child', 'parent', 'bidirectional'}. Specifying child will draw the subgraph toward the child molecules and parent will draw towards lower hierarchy scaffolds. Specifying bidirectional will construct the subgraph in both directions. The default traversal is 'child'.

In [21]:
# Draw graph for scaffold 'c1ccncc1'
widget = visualizer.draw_for_scaffold('c1ccncc1', traversal='child')
widget.layout.height = '600px'
widget

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'nodeSpacing': 50, 'edgeLengthVal': 50}, cytoscape_style=[{…

The layout can also be customised using the `layout_kwargs` argument passing arguments to the CytoscapeWidget.set_layout function. Here we change the algorithm for constructing the layou to 'breadthfirst'.

In [22]:
# Draw graph for scaffold 'c1ccncc1'
widget = visualizer.draw_for_scaffold('c1ccncc1', layout_kwargs={'name': 'breadthfirst'})
widget.layout.height = '600px'
widget

CytoscapeWidget(cytoscape_layout={'name': 'breadthfirst', 'nodeSpacing': 50, 'edgeLengthVal': 50}, cytoscape_s…

A user also has the ability to add further interactivity to the visualisations using the ipywidgets interact function, an example below shows an interactive visualisation where the user can select the molecule to display using a dropdown box and also update the layout.

In [26]:
from ipywidgets import interact, fixed


def draw(visualizer, molecule_id, layout):
    layout_kwargs = {'name': layout}
    widget = visualizer.draw_for_molecule(molecule_id, layout_kwargs)
    widget.layout.height = '600px'
    return widget
    

# Get the list of all molecules in the graph and sort.
mol_ids = sorted(list(tree.get_molecule_nodes()))

# List of a subset of available layouts in cytoscape.
layouts = ['dagre', 'breadthfirst', 'cose']

# Now create the visualization.
interact(
    draw,
    visualizer=fixed(visualizer),
    molecule_id=mol_ids,
    layout=layouts
);

interactive(children=(Dropdown(description='molecule_id', options=('1085650', '1085710', '1106125', '1113907',…